### --- Day 20: Race Condition ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/20

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 20);
WriteLines(inputLines, maxCols: 50);

Loading puzzle file: Day20.txt
Total lines: 141
Max line length: 141

##################################################
#...###.....#...###...#...#...###...###...#.......
#.#.###.###.#.#.###.#.#.#.#.#.###.#.###.#.#.#####.
#.#...#...#...#.....#...#...#...#.#...#.#.#.....#.
#.###.###.#####################.#.###.#.#.#####.#.


Ok, since the cheat is defined by the start / end position. The definition of "save" is relative to the fastest non-cheating time, so we need to find that first.

Then for the cheating part - a two-move cheat that must end back on the path is equivalent to one step through a wall. So perhaps we can try deleting each wall piece and seeing how that affects our path to the end.

This sounds pretty expensive (spoiler: it is), but here's a few points we can leverage to help optimise:

* We can skip wall pieces around the perimeter
* We already know the distance to each point before we start cheating
* Given we know the distance to all points around a wall piece, we know the distance to step into the wall is `min(neighbour distance) + 1`. Consequently we also know the distance for the subsequent step(s).
* Given are only considering 100+ point savings, if cheating through the wall doesn't yield at least 100 point saving for one of the neighbour steps, it would be impossible for the other steps to yield 100+ step savings. So we can stop here!

So for the cheating phase, we can start our shortest-path search from the wall piece, and keep searching so long as we are finding 100+ point savings until we reach the end or run out of steps to take.

In [4]:
string[] testInputLines = [
    "###############",
    "#...#...#.....#",
    "#.#.#.#.#.###.#",
    "#S#...#.#.#...#",
    "#######.#.#.###",
    "#######.#.#...#",
    "#######.#.###.#",
    "###..E#...#...#",
    "###.#######.###",
    "#...###...#...#",
    "#.#####.#.###.#",
    "#.#...#.#.#...#",
    "#.#.#.#.#.#.###",
    "#...#...#...###",
    "###############",
];

In [5]:
using Cost = int;

In [6]:
using PointCost = (Point point, Cost cost);
using SavingCount = (Cost saving, int count);

const char WALL = '#';
const char START = 'S';
const char END = 'E';

static readonly Point[] UDLR = [Up, Down, Left, Right];

IEnumerable<SavingCount> FindCheatSavings(string[] inputLines, int minimumSaving)
{
    CharGrid grid = new(inputLines);
    
    var walls = grid.Enumerate().Where(pch => pch.ch is WALL).Select(pch => pch.point).ToHashSet();
    var start = grid.Enumerate().Where(pch => pch.ch is START).Single().point;
    var end = grid.Enumerate().Where(pch => pch.ch is END).Single().point;

    // Start with the "default" costs, then we can explore a second time and
    // see if we get anything shorter.

    var defaults = ShortestPath(start, GetNextNodeFunc(grid, walls))
                    .ToDictionary(pointCost => pointCost.node, pointCost => pointCost.cost);
    var shortest = defaults[end];
    Console.WriteLine($"Shortest legal path cost is {shortest}");

    // Now that we know the actual shortest cost, find all the candidate edges
    // and try deleting them one-by-one.

    var nonEdgeWalls = walls.Where(wall => (wall) switch {
        (var x, _) when x is 0 || x + 1 == grid.Cols => false,
        (_, var y) when y is 0 || y + 1 == grid.Rows => false,
        _ => true
    }).ToHashSet();

    var savings = nonEdgeWalls
                    .Select(RemoveOne)
                    .Where(diff => diff > 0)
                    .GroupBy(diff => diff)
                    .Select(g => (g.Key, g.Count()));
    return savings;

    Cost RemoveOne(Point wall)
    {
        var wallCost = UDLR
                        .Select(dir => wall + dir)
                        .Where(p => !walls.Contains(p))
                        .Select(p => defaults[p] + 1)
                        .Append(Cost.MaxValue)
                        .Min();

        // Starting from the wall piece we just deleted, see if we can reach the
        // end whilst maintaining our minimum saving
        defaults[wall] = Cost.MaxValue; // Ensure we find a saving for the 0th step
        var (_, newCost) = ShortestPath(wall, GetCheatingNextNode(grid, walls, defaults, wallCost, minimumSaving))
                            .Where(pc => pc.node == end)
                            .FirstOrDefault();
        defaults.Remove(wall);

        // At this point we hopefully made it from the wall to the end
        newCost = newCost switch {
            0 => shortest, // didn't reach the end
            _ => newCost + wallCost // include the steps to reach the wall
        };

        var diff = shortest - newCost;
        return diff;
    }
}

// Standard neighbour search for shortest path algo
NextNodeFunc<Point, Cost> GetNextNodeFunc(CharGrid grid, HashSet<Point> walls)
{
    IEnumerable<PointCost> NextNodeFunc(Point point, Cost cost)
    {
        return UDLR.Select(dir => point + dir)
                    .Where(p => grid.IsValid(p))
                    .Where(p => !walls.Contains(p))
                    .Select(p => (p, cost + 1));
    };

    return NextNodeFunc;
}

// Cheating edition of neighbour search. Find neighbours so long as they are
// still saving us more than `minSaving`
NextNodeFunc<Point, Cost> GetCheatingNextNode(CharGrid grid, HashSet<Point> walls, Dictionary<Point, Cost> defaults, Cost wallCost, Cost minSaving)
{
    var defaultFunc = GetNextNodeFunc(grid, walls);
    PointCost[] none = [];

    IEnumerable<PointCost> NextNodeFunc(Point point, Cost cost)
    {
        var cheatCost = wallCost + cost;
        var defaultCost = defaults[point];

        return (defaultCost - cheatCost - minSaving) switch {
            >= 0 => defaultFunc(point, cost),
            _ => none
        };
    }

    return NextNodeFunc;
}


In [7]:
// For the test input...

// There are 14 cheats that save 2 picoseconds.
// There are 14 cheats that save 4 picoseconds.
// There are 2 cheats that save 6 picoseconds.
// There are 4 cheats that save 8 picoseconds.
// There are 2 cheats that save 10 picoseconds.
// There are 3 cheats that save 12 picoseconds.
// There is one cheat that saves 20 picoseconds.
// There is one cheat that saves 36 picoseconds.
// There is one cheat that saves 38 picoseconds.
// There is one cheat that saves 40 picoseconds.
// There is one cheat that saves 64 picoseconds.

const int testMinSaving = 1;
foreach (var (saving, count) in FindCheatSavings(testInputLines, testMinSaving).OrderBy(x => x.saving))
{
    Console.WriteLine($"There are {count} cheats saving {saving} picos.");
}

Shortest legal path cost is 84
There are 14 cheats saving 2 picos.
There are 14 cheats saving 4 picos.
There are 2 cheats saving 6 picos.
There are 4 cheats saving 8 picos.
There are 2 cheats saving 10 picos.
There are 3 cheats saving 12 picos.
There are 1 cheats saving 20 picos.
There are 1 cheats saving 36 picos.
There are 1 cheats saving 38 picos.
There are 1 cheats saving 40 picos.
There are 1 cheats saving 64 picos.


In [8]:
int CountCheatSavings(string[] inputLines, int minimumSaving)
{
    return FindCheatSavings(inputLines, minimumSaving)
            .Select(x => x.count)
            .Sum();
}

In [9]:
// You aren't sure what the conditions of the racetrack will be like, so to give
// yourself as many options as possible, you'll need a list of the best cheats. How
// many cheats would save you at least 100 picoseconds?

const int part1MinSaving = 100;
var part1Answer = CountCheatSavings(inputLines, part1MinSaving);
Console.WriteLine(part1Answer);

Shortest legal path cost is 9504
1415


In [10]:
// 1415 is correct!
Ensure(1415, part1Answer);